# Neural network Tuning

In [2]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

# part 1 - Tuning batch and epoch

In [3]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [4]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [6]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))

    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [7]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [8]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [9]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/30
39/39 [==============================] - 1s 2ms/step - loss: 7.5102 - accuracy: 0.4010
Epoch 2/30
39/39 [==============================] - 0s 2ms/step - loss: 3.5145 - accuracy: 0.5807
Epoch 3/30
39/39 [==============================] - 0s 2ms/step - loss: 2.2129 - accuracy: 0.5651
Epoch 4/30
39/39 [==============================] - 0s 2ms/step - loss: 1.6628 - accuracy: 0.5339
Epoch 5/30
39/39 [==============================] - 0s 2ms/step - loss: 1.3998 - accuracy: 0.5247
Epoch 6/30
39/39 [==============================] - 0s 2ms/step - loss: 1.2496 - accuracy: 0.5781
Epoch 7/30
39/39 [==============================] - 0s 2ms/step - loss: 1.0735 - accuracy: 0.5938
Epoch 8/30
39/39 [==============================] - 0s 2ms/step - loss: 0.9598 - accuracy: 0.5924
Epoch 9/30
39/39 [==============================] - 0s 2ms/step - loss: 0.8814 - accuracy: 0.6276
Epoch 10/30
39/39 [==============================] - 0s 2ms/step - loss: 0.8029 - accuracy: 0.6380
Epoch 11/30
39/39 [

In [10]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.680990 using {'batch_size': 20, 'epochs': 30}


# part 2- Optimizing Activation Function

In [11]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [12]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [34]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [14]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 1s 2ms/step - loss: 4.8594 - accuracy: 0.5273
Epoch 2/50
77/77 [==============================] - 0s 2ms/step - loss: 2.7004 - accuracy: 0.4909
Epoch 3/50
77/77 [==============================] - 0s 2ms/step - loss: 1.7925 - accuracy: 0.5065
Epoch 4/50
77/77 [==============================] - 0s 2ms/step - loss: 1.2879 - accuracy: 0.5391
Epoch 5/50
77/77 [==============================] - 0s 2ms/step - loss: 1.0307 - accuracy: 0.5898
Epoch 6/50
77/77 [==============================] - 0s 2ms/step - loss: 0.9778 - accuracy: 0.5885
Epoch 7/50
77/77 [==============================] - 0s 2ms/step - loss: 0.8614 - accuracy: 0.6211
Epoch 8/50
77/77 [==============================] - 0s 2ms/step - loss: 0.8171 - accuracy: 0.6419
Epoch 9/50
77/77 [==============================] - 0s 2ms/step - loss: 0.7873 - accuracy: 0.6432
Epoch 10/50
77/77 [==============================] - 0s 2ms/step - loss: 0.7695 - accuracy: 0.6549
Epoch 11/50
77/77 [

In [56]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.708333 using {'optimizer': 'Adam'}


# How can you improve accuracy of the model?

In [35]:
from keras.callbacks import EarlyStopping

In [36]:
early_stopping = EarlyStopping(monitor='accuracy', verbose=1,patience=35, mode='auto')

model = KerasClassifier(build_fn=create_my_model, epochs=100, batch_size=30)

grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y, callbacks=[early_stopping])

Epoch 1/100
26/26 [==============================] - 0s 2ms/step - loss: 1.8918 - accuracy: 0.5182
Epoch 2/100
26/26 [==============================] - 0s 2ms/step - loss: 0.7137 - accuracy: 0.6367
Epoch 3/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.6680
Epoch 4/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6817 - accuracy: 0.6602
Epoch 5/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.6615
Epoch 6/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6662 - accuracy: 0.6667
Epoch 7/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6643 - accuracy: 0.6628
Epoch 8/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6509 - accuracy: 0.6771
Epoch 9/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6508 - accuracy: 0.6667
Epoch 10/100
26/26 [==============================] - 0s 2ms/step - loss: 0.6441 - accuracy: 0.6836
Epoch 11/

In [37]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.660156 using {'optimizer': 'SGD'}
